<a href="https://colab.research.google.com/github/columbia-data-club/meetings/blob/main/2025/february_19_sql_and_nosql_databases_in_python_with_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![A blue background with the pandas logo and the words Columbia Data Club on it](https://raw.githubusercontent.com/columbia-data-club/meetings/main/assets/images/2025/pandas.png)

# SQL and NoSQL Databases in Python with Pandas

February 19, 2025

by [Moacir P. de Sá Pereira](https://moacir.com) for the [Columbia Data Club](https://github.com/columbia-data-club/)


This notebook provides an introduction to SQL and NoSQL databases in general and to using them with pandas in particular. A basic understanding of Python syntax (such as the one covered in the Data Club’s [Intro to Python video](https://youtu.be/l45rzo4MUHs)) should suffice.

The material here is adapted from a [similar Data Club meeting in 2023](https://github.com/columbia-data-club/meetings/blob/main/2023/march_09_python_and_relational_databases.ipynb). That notebook includes code for generating a fake dataset of taxi rides and using the [Pony ORM](https://docs.ponyorm.org/toc.html) to give a Pythonic sheen to using a database.

## What Are SQL and NoSQL?

First, we can begin with what is a database before distinguishing between these two paradigms. Simply, but perhaps unhelpfully, a database is a collection of data where the data are queryable (read: findable) in some fashion.

### SQL (Structured Query Language)

[SQL](https://en.wikipedia.org/wiki/SQL) is a very common language used to query a database. It is older than I am and has been battle tested in much the same way. The language stands in for a kind of paradigm for organizing data, namely into relational databases.

In a relational database, an point of data (a datum) is stored as a row in a table, with properties pertaining to that datum as columns. What makes the database relational is having multiple tables that are linked to each other, so you can have one table for customers, one for orders, and one for products. That way, the entry in the orders table includes a column linking to the appropriate row in the customers table and (this is a simplication) every item in the order links to the appropriate item in the products table.

This structure allows for very fine grained querying, like “show me the ZIP code of every customer who ordered a TK-421 widget in the past three weeks.” This involves hitting all three tables: the products table to find the TK-421 widget, the orders table to find orders in the past three weeks that included the widget, and then the customers table to look up the ZIP codes.

In fact, this is kind of _easy_ to do in SQL.

SQL and relational databases also have a rigid structure and transaction model to ensure that “ACID” is maintained. [ACID](https://en.wikipedia.org/wiki/ACID) (atomicity, consistency, isolation, durability) ensures that databases are in the correct state even if there are power failures or other catastrophes.

### NoSQL (These Days: “Not only SQL”)

On the other hand, [NoSQL](https://en.wikipedia.org/wiki/NoSQL) databases basically cover everything else. Relaxing a bit on the rigid structures of SQL databases, NoSQL databases can be more flexible at the cost of some reliability. They also work great with data that come in with unpredictable structures and can even pretend to be regular relational databases if you like.

NoSQL databases these days come in basically three flavors: [key-value stores](https://en.wikipedia.org/wiki/Key%E2%80%93value_database), [graph databases](https://en.wikipedia.org/wiki/Graph_database), and document stores. We’ll be looking at the latter.

A [document store](https://en.wikipedia.org/wiki/Document-oriented_database) replaces the metaphor of a row in a table linked to rows in other tables with the metaphor of a document in a box of other documents. Some documents might have drawings on them and some might have a receipt stapled to them. The database doesn’t care. It simply indexes whatever comes in and makes the document available for searching later.

Of course, it’s not quite as chaotic as that, but it’s not far off the mark. Typically a document looks somewhat like an object in JavaScript or a dictionary in Python or, well, an XML document. That is, the document has a bunch of properties (keys or tags) that also have values. No two documents in a datastore have to have the same properties, though it is helpful to have _some_ structure to aid in searching.

The loosened ACID requirements on NoSQL make a lot of the Internet possible, because users are able to make changes to a database (like rank a movie in Netflix or post something on Facebook), and the action can happen instantly, with it trickling through the datastores to eventually be consistent. With an SQL database, every like of a movie or post on Facebook would lock up the database server until it could guarantee that the transaction had completed. (Again, an oversimplication, but it's basically right.)

## SQL vs. NoSQL?

There are lots of webpages extolling the virtues of SQL or NoSQL, but hopefully most of them land somewhere in the middle, arguing “it depends.” On the one hand, we can say that systems like online banking would be next to impossible without SQL databases, because of their strictness and reliability. On the other hand, we can say that systems like contemporary social media would be next to impossible without NoSQL databases, because of their flexibility and responsiveness.

In the context of today’s meeting, though, we’ll end up focused on SQL databases, but not before giving a quick example of using the [MongoDB](https://www.mongodb.com/) database in JavaScript. We can use this little bit of code [in OneCompiler](https://onecompiler.com/mongodb/439g5x6kq) to show how things work:

```js
db.fav_foods.insertMany(
  [
    {name: 'Burrito', cuisine: "Mexican", toppings: ["Cheese", "Guacamole"]},
    {name: 'Açaí Bowl', cuisine: "Brazilian", toppings: ["Granola"] },
    {name: 'Pizza', cuisine: "Italian"},
    {name: "Sope", cuisine: "Mexican"}
  ]
);
  
db.fav_foods.find(
  { cuisine: "Mexican" }
);
```

If you’re familiar with JavaScript, this should feel rather immediately legible, and it is not very different in Python.

## Using Baserow as an SQL Database

Below, we’ll be working with a proper SQL database, but I want to take a brief detour here to discuss [Baserow](http://baserow.io), a “no-code” database app in the cloud. If you’ve used Airtable before, it’s a lot like that, but open source and your can run your own version of it on your own machine, which is very handy when you don’t have a budget to deploy a database server on GCP or AWS.

Furthermore, Baserow (like Airtable) is designed to be used with third party clients, which includes connecting to our database from inside this Colab.

In [ ]:
!python -m pip install baserowapi

Let’s set some constants. This token is secret, but I’m going to destroy it after today’s meeting, so future attempts to use this notebook will fail on this section. The table IDs are also unique I think to Baserow.io’s instance of Baserow.

In [ ]:
BASEROW_TOKEN = "mukxURHBRxiFffjdD8vkdH8cYtYo3gaG"
djs_id = 451681
songs_id = 450477

In [ ]:
from baserowapi import Baserow, Filter

# Initialize the Baserow client
baserow = Baserow(url='https://api.baserow.io', token=BASEROW_TOKEN)

# Create a table instance using its ID
djs = baserow.get_table(djs_id)
djs

In [ ]:
[dj.to_dict() for dj in djs.get_rows()]

In [ ]:
dj_name = ""
new_dj = {
    'name': dj_name,
}
#added_row = djs.add_rows(new_dj)

In [ ]:
songs = baserow.get_table(songs_id)
[song.to_dict() for song in songs.get_rows()]

In [ ]:
new_song = {
    "song_name": "Electric Lady",
    "performer": "Janelle Monáe",
    "youtube_url": "https://www.youtube.com/watch?v=LPFgBCUBMYk"
}

#added_song = songs.add_rows(new_song)

How to connect the DJ?

In [ ]:
moacirs = djs.get_rows(filters=[Filter("name", "Moacir")])
moacir_id = [moacir.id for moacir in moacirs][0]
moacir_id

In [ ]:
new_song["dj"] = moacir_id
#added_song = songs.add_rows(new_song)

In [ ]:
[song.to_dict() for song in songs.get_rows()]

## Let’s Get Pokémon Data and Create a Database

Enough Baserow. No free ads. Let’s import an already existing database into our notebook. Brian Radomski and James Allen, building upon the [Veekun Pokédex](https://veekun.com/dex) put together a 14 table database of Pokémon for a course and then [shared it on GitHub](https://github.com/brianr852/Pokemon-Database). We’ve [forked it](https://github.com/columbia-data-club/Pokemon-Database) for safe-keeping. Radomski and Allen include a few reports in their repository and a MySQL dump of the database.

In an ideal world, we would use the [`requests`](https://requests.readthedocs.io/en/latest/) library to download that dump (a database converted into a giant text file with a series of SQL commands that reproduces the database).  Since the dump would be in plain text, we would use the `text` property on the [`Response` object](https://requests.readthedocs.io/en/latest/api/#requests.Response) to read the data in. The code would be straightforward:

```py
import requests

pokemon_database_sql_dump_url = "https://raw.githubusercontent.com/columbia-data-club/Pokemon-Database/master/Dump20160519-1.sql"

r = requests.get(pokemon_database_sql_dump_url)
with open("pokemon_database_sql_dump.sql",'w') as f:
  f.write(r.text)
```

Unfortunately, we cannot use the dump as an actual database without processing it. SQLite can read an `.sql` file and convert it to a database, but it does not look like it’s possible to do that with the Frankenstein SQLite we have in Colab.

Outside of Colab, then, I have already done the conversion. It involved using editing the dump (deleting every instance of `NO_AUTO_CREATE_USER,` which was [removed in MySQL 8.0.11](https://dev.mysql.com/doc/relnotes/mysql/8.0/en/news-8-0-11.html)) and then creating a database:

```sh
mysql.server start
mysql -p pokemon < Dump20160519-1.sql
```

Next, I installed and used the [`mysql-to-sqlite3`](https://pypi.org/project/mysql-to-sqlite3/) Python tool to convert the new MySQL database to a SQLite database:

```sh
pip install mysql-to-sqlite3
mysql2sqlite -u root -p -d pokemon -f pokemon.sqlite
```

This creates a binary `.sqlite` file. I added it to our [Meetings](https://github.com/columbia-data-club/meetings) repository, and we can download and use that, instead.

In [ ]:
import pandas as pd
import requests

pokemon_database_url = "https://github.com/columbia-data-club/meetings/raw/main/assets/data/pokemon.sqlite"

r = requests.get(pokemon_database_url) # create HTTP response object
with open("pokemon.sqlite","wb") as f:
  f.write(r.content)

Next, there’s some kind of new setting that even makes the above, which worked in 2023, crash. So let’s add a magic line of code.

In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

Now, we can use `sqlite` to point to that database file and create an object that exists inside colab. This means we don’t need a database server and can keep everything local to our Colab sandbox. `SQLite` is available as the [`sql` magic extension](https://github.com/catherinedevlin/ipython-sql) in Colab.

In [ ]:
%load_ext sql
%sql sqlite:///pokemon.sqlite

Now we can run some basic SQL queries and make sure the database behaves like we hope it would. Paradoxically, we have a somewhat limited ability to access the metadata for the database. However, we can use the database entity relationship diagram provided by the Radomski and Allen to ground how we can reason about this Pokémon database.

![The entity relationship diagram for the Pokémon database](https://raw.githubusercontent.com/columbia-data-club/meetings/main/assets/images/pokemon-db-erd.png)

Let’s use the `PRAGMA` command to get information about the `pokemon` table.

In [ ]:
%sql PRAGMA table_info([pokemon]);

We can run some of the SQL commands the database’s authors provide to query the data, for example, to get a list of pokémon with attack, defense, and hp greater than 100. The resulting object, `strong_pokemon`, can also be converted to a pandas dataframe.

In [ ]:
%%sql strong_pokemon << SELECT pokemon.pok_id, pokemon.pok_name,
  base_stats.b_atk, base_stats.b_def, base_stats.b_hp,
  base_stats.b_speed, base_stats.b_sp_atk, base_stats.b_sp_def
FROM pokemon
INNER JOIN base_stats
ON pokemon.pok_id = base_stats.pok_id
WHERE b_atk > 100 and b_def > 100 and b_hp > 100
GROUP BY pokemon.pok_id;

In [ ]:
# As an ipython-sql result
strong_pokemon

In [ ]:
# As a dataframe
strong_pokemon_df = strong_pokemon.DataFrame()
strong_pokemon_df

In [ ]:
%%sql pokemon << SELECT
    p.pok_id AS pokemon_id,
    p.pok_name AS name,
    p.pok_height AS height,
    p.pok_weight AS weight,
    bs.b_atk AS attack,
    bs.b_def AS defense,
    bs.b_hp AS HP,
    bs.b_speed AS speed,
    bs.b_sp_atk AS special_attack,
    bs.b_sp_def AS special_defense,
    GROUP_CONCAT(a.abil_name, ', ') AS abilities
FROM
    pokemon p
INNER JOIN
    base_stats bs ON p.pok_id = bs.pok_id
INNER JOIN
    pokemon_abilities pa ON p.pok_id = pa.pok_id
INNER JOIN
    abilities a ON pa.abil_id = a.abil_id
GROUP BY
    p.pok_id, p.pok_name;

In [ ]:
pokemon

In [ ]:
df = pokemon.DataFrame()
df["abilities"] = df["abilities"].str.split(", ")
abilities_encoded = df['abilities'].str.join('|').str.get_dummies()
df = pd.concat([df.drop(columns=['abilities']), abilities_encoded], axis=1)

In [ ]:
df.head()

In [ ]:
df[df["name"] == "pikachu"]

In [ ]:
%%sql abilities << SELECT
abil_name
from abilities

In [ ]:
abilities

## OMG This Is ETL Right?

In converting parts of the database into a dataframe, we are doing something like extracting data in the [ETL](https://en.wikipedia.org/wiki/Extract,_transform,_load) (extract, transform, load) workflow in data engineering.

Here are two books about ETL:

* _Understanding ETL_ by Matt Palmer (O’Reilly but free I think [if you give Databricks your email](https://www.databricks.com/resources/ebook/understanding-etl))
* _Fundamentals of Data Engineering_ by Joe Reis and Matt Housley ([O’Reilly](https://www.oreilly.com/library/view/fundamentals-of-data/9781098108298/))

Unfortunately, Columbia Libraries has neither.

We’ll be looking into ETL in more detail (namely the “transform” part) in the next two Data Club meetings, but it’s important to see that what we did here is a kind of extraction, especially:

In [74]:
df.to_csv("pokemon.csv")